## Analysing Results for SCHISM

A workflow for reading and visualizing data.

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
from pyPoseidon.utils import data,point
from pyPoseidon.utils.vis import video
import numpy as np
import xarray as xr
import glob
import hvplot.pandas
import hvplot.xarray

In [ ]:
pyPoseidon.__version__

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
import cartopy.crs as ccrs

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

## Analysis of output 

In [ ]:
PATH = '/Users/brey/SCHISM/test_grid_2/'

In [ ]:
folders = glob.glob(PATH + '*')
folders

In [ ]:
folders=[PATH]
folders

In [ ]:
otp = data(**{'solver':'schism','folders': folders})#,'savenc':True})#,tag='eur025')# optional tag for data selection ,sa_date='2010-2-1 5:0:0',se_date='2010-2-1 22:0:0') # optionally set range for tide gauge data acquisition 

In [ ]:
otp.impl.Dataset

In [ ]:
otp.impl.Dataset.SCHISM_hgrid_edge_nodes

### Visualize

In [ ]:
otp.impl.Dataset.splot.contourf('depth' ,vmin=otp.impl.Dataset.depth.min(), vmax=otp.impl.Dataset.depth.max())

In [ ]:
otp.impl.Dataset.splot.grid()

In [ ]:
otp.impl.Dataset.splot.contourf('elev',it=4,scale=.05)

In [ ]:
v = otp.impl.Dataset.splot.frames('elev',title='SSH')#,vmin=-.5, vmax=.5, nv=20)
v

In [ ]:
v.save('/Users/brey/Downloads/ssh.mp4', writer='ffmpeg')

In [ ]:
video('/Users/brey/Downloads/ssh.mp4','mp4') # optional show mp4 video

In [ ]:
w = otp.impl.Dataset.splot.qframes('wind_speed',title='Vel',scale=30.)
w

In [ ]:
w.save('/Users/brey/Downloads/uv.mp4', writer='ffmpeg')

In [ ]:
video('/Users/brey/Downloads/uv.mp4','mp4') # optional show mp4 video

## Observation points / Validation

#### Numerical simulation time series

In [ ]:
otp.impl.info['obs'].head()

In [ ]:
files = [folder +'/trih-'+otp.impl.info['tag']+'.nc' for folder in folders]
files

In [ ]:
op = xr.open_mfdataset(files)

In [ ]:
XS = op.XSTAT[0,:].values
YS = op.YSTAT[0,:].values

In [ ]:
op

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

otp.impl.dem.bathymetry.plot(ax=ax)

ax.scatter(XS,YS,marker='X')

ax.coastlines('50m'); ax.gridlines(draw_labels=True);

In [ ]:
op

In [ ]:
op.ZWL[:,0]

#### Tide gauge data

In [ ]:
otp.impl.obs.locations

In [ ]:
# Get with index number
p = otp.impl.obs.iloc(0)
print (p.head())
print (p.tail())

In [ ]:
#get with Station Name  
p = otp.impl.obs.loc('Denmark - Faaborg')
p.head()

In [ ]:
#plot
ax = p.plot()

### sample nearest point from simulation 

In [ ]:
plat,plon = otp.impl.obs.locations.loc[0,['lat','lon']]
plat, plon

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.subplot(projection=ccrs.PlateCarree());

gp = ax.contourf(otp.impl.dem.longitude, otp.impl.dem.latitude, otp.impl.dem.bathymetry, transform=ccrs.PlateCarree());

pp = ax.scatter(plon, plat, marker='x', s=50, color='r', label='obs locations', transform=ccrs.PlateCarree());



In [ ]:
#ts = otp.vars.S1.sel(XZ=[lon], method='nearest', tolerance=5).sel(YZ=[lat], method='nearest', tolerance=5)
#ts = ts.squeeze().to_pandas()
#ts.head()

In [ ]:
ts = point(lon=plon,lat=plat,data=otp.impl)
ts.tseries(var='S1')

In [ ]:
ts.S1.head()

In [ ]:
ts.S1.plot()

In [ ]:
## Join the graphs
ax = p.plot(figsize=(14,10))
ts.S1.plot(ax=ax,color=['r'])
ax.legend(list(p.columns.values) + ['D3D'], loc='center left',bbox_to_anchor=(1.1, 0.5))

...or...

In [ ]:
## Join the graphs
p.hvplot(figsize=(14,10)) * ts.S1.hvplot(label='d3d',color='red')